# Coverage performance analysis

In [ ]:
# Just for line magics exceding the line length:
# ruff: noqa: E501

# Some notebook working parameters that can be changed:
do_time_comparisions = True
do_profiles = True
time_loops = 10

available_aoi_files = [
    "../single_aoi.geojson",
    "single_aoi_crosses_antimeridian.geojson",
    "single_aoi_crosses_antimeridian_counter.geojson",
]
available_aoi_files_index = 0

In [ ]:
import ephemerista
from ephemerista.analysis.coverage import Coverage
from ephemerista.analysis.visibility import Visibility  # noqa: F401
from ephemerista.assets import Asset, GroundStation, Spacecraft  # noqa: F401
from ephemerista.propagators.sgp4 import SGP4
from ephemerista.scenarios import Scenario
from ephemerista.time import TimeDelta

In [ ]:
%%time
ephemerista.init(
    eop_path="../../tests/resources/finals2000A.all.csv",
    spk_path="../../tests/resources/de440s.bsp",
)

In [ ]:
%%time
tle = """SENTINEL-6
1 46984U 20086A   24319.21552651 -.00000061  00000+0  71254-6 0  9995
2 46984  66.0411 259.6585 0007844 270.2444  89.7673 12.80930600186045
"""

propagator = SGP4(tle=tle)
sc = Asset(model=Spacecraft(propagator=propagator), name="PHASMA")

In [ ]:
%%time
start_time = propagator.time
end_time_one_orbit = start_time + TimeDelta.from_minutes(112)
end_time_five_days = start_time + TimeDelta.from_hours(24 * 5)

# Comparing hexagonal and rectangle region polygonization

The `polygonize_aoi` method takes an additional argument `min_elevation_deg` to define at which minimum elevation from the ground locations the visibility must be computed.

In this example we use 70° minimum elevation to model an optical satellite which can depoint a bit off-nadir but not too much in order not to loose too much image quality.

In [ ]:
import geojson_pydantic

from ephemerista.scenarios import polygonize_aoi, polygonize_aoi_rectangles

if do_profiles:
    %load_ext line_profiler

In [ ]:
with open(available_aoi_files[available_aoi_files_index]) as f:
    aoi = geojson_pydantic.FeatureCollection.model_validate_json(f.read())

In [ ]:
def plot_feature_list(
    aoi: list | None = None,
    feature_list: list | None = None,
    title="AOI Sub-Polygons",
    only_indexes: list[int] | None = None,
):
    """
    Plot a list of polygon features using matplotlib and shapely.
    """
    import matplotlib.pyplot as plt
    from shapely.geometry import shape

    plt.figure(figsize=(16, 6))
    if aoi:
        for feature in aoi:
            geom = shape(feature.geometry)
            x, y = geom.exterior.xy
            plt.plot(x, y, color="lime", linewidth=3)

    if feature_list:
        index = -1
        for feature in feature_list:
            index = index + 1
            if only_indexes and index not in only_indexes:
                continue

            geom = shape(feature.geometry)
            x, y = geom.exterior.xy
            plt.plot(x, y, marker="o")

    plt.title(title)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.show()

The following Figure shows the area defined by the AOI and how the polygons cover it. It can be misleading for the case of AOIs crossing the antimeridian, as they are [ploted as if it goes the other way around the globe](https://www.gadom.ski/antimeridian/latest/#whats-the-problem).

In [ ]:
plot_feature_list(aoi=aoi.features, title="AOI features")

In [ ]:
%%time
if do_time_comparisions:
    for _i in range(time_loops):
        feature_list_hex = polygonize_aoi(
            aoi_geom_dict=aoi.__geo_interface__["features"][0]["geometry"],
            res=1,
            min_elevation_deg=70.0,
        )
else:
    feature_list_hex = polygonize_aoi(
        aoi_geom_dict=aoi.__geo_interface__["features"][0]["geometry"],
        res=1,
        min_elevation_deg=70.0,
    )

In [ ]:
if do_profiles:
    %lprun  -u 1 -f polygonize_aoi polygonize_aoi(aoi_geom_dict=aoi.__geo_interface__["features"][0]["geometry"], res=1, min_elevation_deg=70.0)

In [ ]:
%%time
if do_time_comparisions:
    for _i in range(time_loops):
        feature_list_rectangle = polygonize_aoi_rectangles(
            aoi_geom_dict=aoi.__geo_interface__["features"][0]["geometry"],
            vertex_degrees=6,
            min_elevation_deg=70.0,
        )
else:
    feature_list_rectangle = polygonize_aoi_rectangles(
        aoi_geom_dict=aoi.__geo_interface__["features"][0]["geometry"],
        vertex_degrees=6,
        min_elevation_deg=70.0,
    )

In [ ]:
if do_profiles:
    %lprun  -u 1 -f polygonize_aoi_rectangles polygonize_aoi_rectangles(aoi_geom_dict=aoi.__geo_interface__["features"][0]["geometry"], vertex_degrees=6, min_elevation_deg=70.0)

In [ ]:
plot_feature_list(aoi=aoi, feature_list=feature_list_hex, title="feature_list_hex")
plot_feature_list(aoi=aoi, feature_list=feature_list_rectangle, title="feature_list_rectangle")

## Scenarios

In [ ]:
def get_scenario(feature_list, start_time=start_time, end_time=end_time_one_orbit):
    return Scenario(
        assets=[sc],
        name="Coverage analysis",
        start_time=start_time,
        end_time=end_time,
        areas_of_interest=feature_list,
    )

In [ ]:
def get_coverage_results(scenario):
    cov_one_orbit = Coverage(scenario=scenario)
    results_one_orbit = cov_one_orbit.analyze()
    return cov_one_orbit, results_one_orbit

### One orbit scenario (hexagonal polygons)

In [ ]:
%%time
scenario_one_orbit_hex = get_scenario(feature_list_hex)

In [ ]:
%%time
cov_one_orbit_hex, results_one_orbit_hex = get_coverage_results(scenario_one_orbit_hex)

In [ ]:
if do_profiles:
    %lprun -u 1 -f Coverage.analyze -f Visibility.analyze cov_one_orbit_hex.analyze()

### One orbit scenario (rectangles)

In [ ]:
%%time
scenario_one_orbit_rectangle = get_scenario(feature_list_rectangle)

In [ ]:
%%time
cov_one_orbit_rectangle, results_one_orbit_rectangle = get_coverage_results(scenario_one_orbit_rectangle)

In [ ]:
if do_profiles:
    %lprun  -u 1 -f Coverage.analyze -f GroundStation.propagate -f Visibility.analyze -f Spacecraft.propagate cov_one_orbit_rectangle.analyze()

### 5 days orbit scenario (hexagonal polygons)

In [ ]:
%%time
scenario_five_days_hex = get_scenario(feature_list_hex, end_time=end_time_five_days)

In [ ]:
%%time
cov_five_days_hex, results_five_days_hex = get_coverage_results(scenario_five_days_hex)

In [ ]:
if do_profiles:
    %lprun -u 1 -f Coverage.analyze cov_five_days_hex.analyze()

### 5 days orbit scenario (rectangles)

In [ ]:
%%time
scenario_five_days_rectangle = get_scenario(feature_list_rectangle, end_time=end_time_five_days)

In [ ]:
%%time
cov_five_days_rectangle, results_five_days_rectangle = get_coverage_results(scenario_five_days_rectangle)

In [ ]:
if do_profiles:
    %lprun -u 1 -f Coverage.analyze cov_five_days_rectangle.analyze()

## display results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
display(results_one_orbit_hex.to_geodataframe())
display(results_one_orbit_rectangle.to_geodataframe())
display(results_five_days_hex.to_geodataframe())
display(results_five_days_rectangle.to_geodataframe())

In [ ]:
ax = results_one_orbit_hex.plot_mpl(legend=True, cmap="viridis")
plt.show()
ax = results_one_orbit_rectangle.plot_mpl(legend=True, cmap="viridis")
plt.show()
ax = results_five_days_hex.plot_mpl(legend=True, cmap="viridis")
plt.show()
ax = results_five_days_rectangle.plot_mpl(legend=True, cmap="viridis")
plt.show()

In [ ]:
fig = results_one_orbit_hex.plot_plotly(color_continuous_scale="Jet", opacity=0.5)
fig.show()
fig = results_one_orbit_rectangle.plot_plotly(color_continuous_scale="Jet", opacity=0.5)
fig.show()
fig = results_five_days_hex.plot_plotly(color_continuous_scale="Jet", opacity=0.5)
fig.show()
fig = results_five_days_rectangle.plot_plotly(color_continuous_scale="Jet", opacity=0.5)
fig.show()